In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit

# Load the data
df = pd.read_csv("data_draft_trades_m24_Chris.csv")

In [35]:
df_raw = df.copy().iloc[:,:12]
df_raw.head()

,year,teamup,upick1,upick2,upick3,teamdown,dpick1,dpick2,dpick3,dpick4,dpick5,dpick6
0,2006,Rams,1-11,NaN,NaN,Broncos,1-15,3-68,NaN,NaN,NaN,NaN
1,2006,Ravens,1-12,NaN,NaN,Browns,1-13,6-181,NaN,NaN,NaN,NaN
2,2006,49ers,1-22,NaN,NaN,Broncos,2-37,3-68,NaN,NaN,NaN,NaN
3,2006,Steelers,1-25,NaN,NaN,Giants,1-32,3-96,4-129,NaN,NaN,NaN
4,2006,Bills,1-26,NaN,NaN,Bears,2-42,3-73,NaN,NaN,NaN,NaN


In [37]:
df_raw.keys()

Index(['year', 'teamup', 'upick1', 'upick2', 'upick3', 'teamdown', 'dpick1',
       'dpick2', 'dpick3', 'dpick4', 'dpick5', 'dpick6'],
      dtype='object')

In [25]:
df_processed = pd.DataFrame(np.full(shape=(487,7),fill_value=np.nan),
                            columns=['Year','TeamUp','UpPicks','TeamDown','DownPicks','FutureUpPicks','FutureDownPicks'])

In [38]:
df_processed['Year'] = df_raw['year']
df_processed['TeamUp'] = df_raw['teamup']
df_processed['TeamDown'] = df_raw['teamdown']

In [39]:
df_processed.head()

,Year,TeamUp,UpPicks,TeamDown,DownPicks,FutureUpPicks,FutureDownPicks
0,2006,Rams,NaN,Broncos,NaN,NaN,NaN
1,2006,Ravens,NaN,Browns,NaN,NaN,NaN
2,2006,49ers,NaN,Broncos,NaN,NaN,NaN
3,2006,Steelers,NaN,Giants,NaN,NaN,NaN
4,2006,Bills,NaN,Bears,NaN,NaN,NaN


In [68]:
def process_up_picks():
    up_cols = df_raw.iloc[:, 2:5]
    
    cleaned_picks = up_cols.applymap(
        lambda x: str(x).split('-')[-1] if pd.notna(x) else None
    )

    combined_picks = cleaned_picks.apply(lambda row: ";".join(row.dropna()), axis=1)

    return combined_picks
    
def process_down_picks():
    down_cols = df_raw.iloc[:, 7:]
    
    
    cleaned_picks = down_cols.applymap(
        lambda x: str(x).split('-')[-1] if pd.notna(x) else None
    )

    combined_picks = cleaned_picks.apply(lambda row: ";".join(row.dropna()), axis=1)

    return combined_picks

In [69]:
df_processed['UpPicks'] = process_up_picks()
df_processed.head(20)

/var/folders/ph/1mrmk6rd0x39gd0qq7yhcbkc0000gn/T/ipykernel_20514/1045192589.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_picks = up_cols.applymap(


,Year,TeamUp,UpPicks,TeamDown,DownPicks,FutureUpPicks,FutureDownPicks
0,2006,Rams,11,Broncos,68,NaN,NaN
1,2006,Ravens,12,Browns,181,NaN,NaN
2,2006,49ers,22,Broncos,68,NaN,NaN
3,2006,Steelers,25,Giants,96;129,NaN,NaN
4,2006,Bills,26,Bears,73,NaN,NaN
5,2006,Redskins,35,Jets,189;37,NaN,NaN
6,2006,Patriots,36,Packers,75,NaN,NaN
7,2006,Falcons,37;139,Packers,93;148,NaN,NaN
8,2006,Eagles,39,Titans,116,NaN,NaN
9,2006,Giants,44,Ravens,87,NaN,NaN


In [71]:
df_processed['DownPicks'] = process_down_picks()
df_processed.head(25)

/var/folders/ph/1mrmk6rd0x39gd0qq7yhcbkc0000gn/T/ipykernel_20514/1045192589.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_picks = down_cols.applymap(


,Year,TeamUp,UpPicks,TeamDown,DownPicks,FutureUpPicks,FutureDownPicks
0,2006,Rams,11,Broncos,68,NaN,NaN
1,2006,Ravens,12,Browns,181,NaN,NaN
2,2006,49ers,22,Broncos,68,NaN,NaN
3,2006,Steelers,25,Giants,96;129,NaN,NaN
4,2006,Bills,26,Bears,73,NaN,NaN
5,2006,Redskins,35,Jets,189;37,NaN,NaN
6,2006,Patriots,36,Packers,75,NaN,NaN
7,2006,Falcons,37;139,Packers,93;148,NaN,NaN
8,2006,Eagles,39,Titans,116,NaN,NaN
9,2006,Giants,44,Ravens,87,NaN,NaN


In [89]:
df_processed.replace('', 0, inplace=True)

In [90]:
df_processed.head(25)

,Year,TeamUp,UpPicks,TeamDown,DownPicks,FutureUpPicks,FutureDownPicks,l1_value,l2_value
0,2006,Rams,11,Broncos,68,NaN,NaN,4.199157e-01,1.763292e-01
1,2006,Ravens,12,Browns,181,NaN,NaN,3.900722e-01,1.521563e-01
2,2006,49ers,22,Broncos,68,NaN,NaN,1.663903e-01,2.768572e-02
3,2006,Steelers,25,Giants,96;129,NaN,NaN,1.323855e-01,1.752591e-02
4,2006,Bills,26,Bears,73,NaN,NaN,1.200842e-01,1.442021e-02
5,2006,Redskins,35,Jets,189;37,NaN,NaN,-4.134720e-02,1.709591e-03
6,2006,Patriots,36,Packers,75,NaN,NaN,5.204704e-02,2.708894e-03
7,2006,Falcons,37;139,Packers,93;148,NaN,NaN,9.924805e-02,9.850175e-03
8,2006,Eagles,39,Titans,116,NaN,NaN,4.258719e-02,1.813669e-03
9,2006,Giants,44,Ravens,87,NaN,NaN,2.757555e-02,7.604110e-04


In [91]:
def weibull_value(x,lambda_, beta_):
    return np.exp(-lambda_ * ((x-1)** beta_))

def analyze_trade_l1(row):
    init_lambda_ = 0.090810427377139
    init_beta_ = 0.9752792185420724

    up_picks = np.array([int(p) for p in str(row['UpPicks']).split(';') if p])
    down_picks = np.array([int(p) for p in str(row['DownPicks']).split(';') if p])

    up_picks_values = np.array([weibull_value(up_picks, init_lambda_, init_beta_) for p in up_picks])
    down_picks_values = np.array([weibull_value(down_picks, init_lambda_, init_beta_) for p in down_picks])

    return np.sum(up_picks_values) - np.sum(down_picks_values)

def analyze_trade_l2(row):
    init_lambda_ = 0.090810427377139
    init_beta_ = 0.9752792185420724

    up_picks = np.array([int(p) for p in str(row['UpPicks']).split(';') if p])
    down_picks = np.array([int(p) for p in str(row['DownPicks']).split(';') if p])

    up_picks_values = np.array([weibull_value(up_picks, init_lambda_, init_beta_) for p in up_picks])
    down_picks_values = np.array([weibull_value(down_picks, init_lambda_, init_beta_) for p in down_picks])

    return (np.sum(up_picks_values) - np.sum(down_picks_values))**2

df_processed['l1_value'] = df_processed.apply(analyze_trade_l1, axis=1)
df_processed['l2_value'] = df_processed.apply(analyze_trade_l2, axis=1)


/var/folders/ph/1mrmk6rd0x39gd0qq7yhcbkc0000gn/T/ipykernel_20514/919342725.py:2: RuntimeWarning: invalid value encountered in power
  return np.exp(-lambda_ * ((x-1)** beta_))


In [92]:
df_processed.head(25)

,Year,TeamUp,UpPicks,TeamDown,DownPicks,FutureUpPicks,FutureDownPicks,l1_value,l2_value
0,2006,Rams,11,Broncos,68,NaN,NaN,4.199157e-01,1.763292e-01
1,2006,Ravens,12,Browns,181,NaN,NaN,3.900722e-01,1.521563e-01
2,2006,49ers,22,Broncos,68,NaN,NaN,1.663903e-01,2.768572e-02
3,2006,Steelers,25,Giants,96;129,NaN,NaN,1.323855e-01,1.752591e-02
4,2006,Bills,26,Bears,73,NaN,NaN,1.200842e-01,1.442021e-02
5,2006,Redskins,35,Jets,189;37,NaN,NaN,-4.134720e-02,1.709591e-03
6,2006,Patriots,36,Packers,75,NaN,NaN,5.204704e-02,2.708894e-03
7,2006,Falcons,37;139,Packers,93;148,NaN,NaN,9.924805e-02,9.850175e-03
8,2006,Eagles,39,Titans,116,NaN,NaN,4.258719e-02,1.813669e-03
9,2006,Giants,44,Ravens,87,NaN,NaN,2.757555e-02,7.604110e-04
